In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots

In [ ]:
from topo import SQuad

In [ ]:
order   = 2
L       = 2*np.pi
n_elems = 2

x_max = L
y_max = L/2.0
x_vals = np.linspace(0, x_max, n_elems+1)
y_vals = np.linspace(0, y_max, n_elems+1)

vertices = np.zeros(((n_elems+1)**2, 2), dtype=np.double)
elem_to_vertex = np.zeros((n_elems**2, 4), dtype=np.int)

for i in range(n_elems):
    for j in range(n_elems):
        elem = i*n_elems+j
        elem_to_vertex[elem,0] = i*(n_elems+1)+j
        elem_to_vertex[elem,1] = i*(n_elems+1)+j+1
        elem_to_vertex[elem,2] = (i+1)*(n_elems+1)+j+1
        elem_to_vertex[elem,3] = (i+1)*(n_elems+1)+j

for i in range(n_elems+1):
    for j in range(n_elems+1):
        v = i*(n_elems+1)+j
        vertices[v,0] = x_vals[j]
        vertices[v,1] = y_vals[i]


In [ ]:
topo  = SQuad()
nodes = vertices[elem_to_vertex]
jacb  = topo.calc_jacb(nodes)
jacb_inv = topo.calc_jacb_inv(jacb)

In [ ]:
ref = np.array([[0.0, 0.0],
                [.75,.75],
                [.75,-.75],
                [-.75,.75],
                [-.75,-.75],], dtype=np.double)
#ref = np.array([[0.0, 0.0]], dtype=np.double)
#topo.ref_to_phys(nodes, jacb, ref)

In [ ]:
phys = topo.ref_to_phys(nodes, jacb, ref)

In [ ]:
plt.scatter(phys[:,:,0], phys[:,:,1])

for x in x_vals:
    plt.vlines(x, 0, y_max)
for y in y_vals:
    plt.hlines(y, 0, x_max)

In [ ]:
plt.scatter(ref[:,0], ref[:,1])
plt.vlines(1, -1, 1)
plt.vlines(-1, -1, 1)
plt.hlines(1, -1, 1)
plt.hlines(-1, -1, 1)